In [1]:
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab, CatTargetDriftTab

In [2]:
from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection, CatTargetDriftProfileSection 

In [3]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [4]:
#df = pd.read_csv("/Users/rsh/Desktop/topic_modeling/local/internal_model.csv")
file_path = "data_with_embeddings_22000.csv"
df = pd.read_csv(file_path)

In [5]:
new_data_path = "sampled_data_with_embeddings_testset.csv"
df_new_data = pd.read_csv(new_data_path)

In [6]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'entry_id', 'excerpt',
       'analysis_framework_id', 'lead_id', 'project_id', 'verified', 'sectors',
       'subpillars_2d', 'subpillars_1d', 'geo_location',
       'specific_needs_groups', 'severity', 'info_date', 'reliability',
       'affected_groups_level_0', 'affected_groups_level_1',
       'affected_groups_level_2', 'affected_groups_level_3', 'age', 'gender',
       'source_type', 'url', 'website', 'lang', 'translation_en',
       'translation_fr', 'translation_es', 'embeddings'],
      dtype='object')

In [7]:
df_new_data.columns

Index(['Unnamed: 0', 'entry_id', 'excerpt', 'analysis_framework_id', 'lead_id',
       'project_id', 'verified', 'sectors', 'subpillars_2d', 'subpillars_1d',
       'geo_location', 'specific_needs_groups', 'severity', 'info_date', 'age',
       'gender', 'reliability', 'affected_groups_level_0',
       'affected_groups_level_1', 'affected_groups_level_2',
       'affected_groups_level_3', 'source_type', 'url', 'website', 'lang',
       'translation_en', 'translation_fr', 'translation_es', 'embeddings'],
      dtype='object')

In [8]:
df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)
df_new_data = df_new_data.drop(["Unnamed: 0"], axis=1)

In [9]:
df.head(2)

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,...,age,gender,source_type,url,website,lang,translation_en,translation_fr,translation_es,embeddings
0,489433,Primary and secondary net enrollment rates are...,1306.0,67488.0,2225.0,False,['Education'],['Humanitarian Conditions->Living Standards'],['Context->Socio Cultural'],['République démocratique du Congo'],...,['Children/Youth (5 to 17 years old)'],[],website,https://blogs.worldbank.org/education/free-pri...,blogs.worldbank.org,en,NaN,Les taux de scolarisation nets primaires et se...,Las tasas de inscripción netas primarias y sec...,"[-0.12575942277908325, 0.011437282897531986, 0..."
1,489439,The government is also tackling several import...,1306.0,67488.0,2225.0,False,['Education'],['Capacities & Response->National Response'],['Context->Socio Cultural'],['République démocratique du Congo'],...,['Children/Youth (5 to 17 years old)'],[],website,https://blogs.worldbank.org/education/free-pri...,blogs.worldbank.org,en,NaN,Le gouvernement s'attaque également à plusieur...,El Gobierno también está abordando varios elem...,"[-0.0262149628251791, 0.0545789934694767, 0.01..."


In [10]:
df_new_data.head(2)

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,...,affected_groups_level_2,affected_groups_level_3,source_type,url,website,lang,translation_en,translation_fr,translation_es,embeddings
0,220648,"[9th Dec, Overall Syria] It is understood that...",1306,45655.0,2028.0,False,[],[],"['Covid-19->Deaths', 'Covid-19->Cases']",[],...,[],[],website,https://reliefweb.int/sites/reliefweb.int/file...,reliefweb.int,en,NaN,"[9ème décembre, la Syrie globale] Il est enten...","[9 de diciembre, Siria general] Se entiende qu...","[-0.021705863997340202, 0.02024352177977562, -..."
1,492314,"[August 2021, Overall Syria] In light of the d...",1306,67914.0,2028.0,False,"['Cross', 'Livelihoods']","['Impact->Impact On Systems, Services And Netw...",[],"['Hama', 'Syrian Arab Republic', 'Damascus', '...",...,"['Displaced', 'Non Displaced', 'Non Displaced'...","['None', 'Host', 'IDP']",website,https://fscluster.org/sites/default/files/docu...,fscluster.org,en,NaN,"[Août 2021, Syrie globale] À la lumière de la ...","[De agosto de 2021, Siria general] A la luz de...","[-0.20314602553844452, 0.003621827345341444, 0..."


In [11]:
df["project_id"].unique()

array([2225., 2170., 1183., 2311., 2028., 2098., 2099., 2335., 2334.,
       2336., 2331.])

In [12]:
df_new_data["project_id"].unique()

array([2028., 2098., 2099., 2170., 2225., 2311.])

In [13]:
def use_mask(df, category, tag_name, project_id=None):
    #df[category] = df[category].apply(eval)
    if project_id:
        mask = (df[category].apply(eval).apply(lambda x: tag_name in x)) & (df["project_id"] == project_id)
    else:
        mask = df[category].apply(eval).apply(lambda x: tag_name in x)
    df_masked = df[mask]
    return df_masked

In [14]:
def use_mask_project_id_based(df, project_id=None):
    if project_id:
        mask = df["project_id"] == project_id
    
    df_masked = df[mask]
    return df_masked

In [15]:
def create_embedding_df(df):
    df_new = df["embeddings"].apply(eval).apply(np.array)
    embedding_list = df_new.to_list()
    return embedding_list

In [16]:
#df_reference = use_mask(df, "sectors", "Nutrition", 2225.0)
#df_newdata = use_mask(df, "sectors", "Food Security", 1898.0)

df_reference = use_mask_project_id_based(df, 2028.0)
df_newdata = use_mask_project_id_based(df_new_data, 2170.0)

In [17]:
len(df_reference), len(df_newdata)

(2000, 1000)

In [18]:
df_reference.head(2)

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,...,age,gender,source_type,url,website,lang,translation_en,translation_fr,translation_es,embeddings
12,489889,"[16 September, 2021, GoS] First Pakistan Inter...",1306.0,67778.0,2028.0,False,['Logistics'],['Humanitarian Conditions->Living Standards'],[],['Government-Controlled Areas'],...,[],[],website,http://sana.sy/en/?p=248722,http://sana.sy/en/,en,NaN,"[16 septembre, 2021, GOS] Le premier vol de Pa...","[16 de septiembre, 2021, GOS] El primer vuelo ...","[0.05191599205136299, -0.03794324770569801, 0...."
20,228408,"[November 2020, NWS] November findings highlig...",1306.0,46537.0,2028.0,False,"['Health', 'Shelter', 'Education']","['Humanitarian Conditions->Living Standards', ...",[],"['Syrian Arab Republic', 'Northwest']",...,[],[],website,https://www.impact-repository.org/document/rea...,https://www.impact-repository.org/,en,NaN,"[Novembre 2020, NWS] Les conclusions de novemb...","[De noviembre de 2020, NWS] Los hallazgos de n...","[-0.09823006391525269, 0.06384459882974625, -0..."


In [19]:
df_newdata.head(2)

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,...,affected_groups_level_2,affected_groups_level_3,source_type,url,website,lang,translation_en,translation_fr,translation_es,embeddings
3000,489909,Education: Needs: • Heavy windstorms and flood...,1306,67788.0,2170.0,False,['Education'],"['Impact->Impact On Systems, Services And Netw...",[],['Adamawa'],...,[],[],website,https://reliefweb.int/sites/reliefweb.int/file...,www.reliefweb.int,en,NaN,Education: [interprétation] Besoints: • Des te...,Educación: Necesidades: • Las tormentas pesada...,"[-0.04977746307849884, 0.05154505372047424, 0...."
3001,393645,"[7th Apr 2021, Borno State] Direct observation...",1306,62284.0,2170.0,False,['WASH'],['Humanitarian Conditions->Living Standards'],[],['Borno'],...,[],[],disk,NaN,NaN,en,NaN,"[7ème avril 2021, État Borno] Les observations...","[7 de abril de 2021, estado BORNO] Las observa...","[-0.09627554565668106, 0.08278235793113708, 0...."


In [20]:
embedding_lst_reference = create_embedding_df(df_reference)

In [21]:
reference_df = pd.DataFrame(embedding_lst_reference)

In [22]:
embedding_lst_newdata = create_embedding_df(df_newdata)

In [23]:
newdata_df = pd.DataFrame(embedding_lst_newdata)

In [24]:
len(reference_df), len(newdata_df)

(2000, 1000)

In [25]:
data_drift_profile = Profile(sections=[DataDriftProfileSection()])

In [60]:
if len(reference_df) and len(newdata_df):
    data_drift_profile.calculate(reference_df[:1000], newdata_df[:1000], column_mapping=None) 
else:
    print("One of the dfs have no data")

In [61]:
data_drift_report_json = json.loads(data_drift_profile.json())

In [62]:
data_drift_report_json["data_drift"]["data"]["options"]

{'confidence': None, 'drift_share': 0.5, 'nbinsx': 10, 'xbins': None}

In [63]:
print(f"n_features: {data_drift_report_json['data_drift']['data']['metrics']['n_features']}")
print(f"n_drifted_features: {data_drift_report_json['data_drift']['data']['metrics']['n_drifted_features']}")
print(f"share_drifted_features: {data_drift_report_json['data_drift']['data']['metrics']['share_drifted_features']}")
print(f"dataset_drift: {data_drift_report_json['data_drift']['data']['metrics']['dataset_drift']}")


n_features: 768
n_drifted_features: 616
share_drifted_features: 0.8020833333333334
dataset_drift: True


In [64]:
data_drift_report_json

{'data_drift': {'name': 'data_drift',
  'datetime': '2022-12-02 11:57:08.969681',
  'data': {'utility_columns': {'date': None,
    'id': None,
    'target': None,
    'prediction': None},
   'cat_feature_names': [],
   'num_feature_names': [0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    92,
    93,
    94,
    95,
    9

In [16]:
data_drift_profile = Profile(sections=[DataDriftProfileSection()])

In [17]:
reference_project_ids = list(df["project_id"].unique())
new_project_ids = list(df_new_data["project_id"].unique())

In [18]:
reference_project_ids, new_project_ids

([2225.0,
  2170.0,
  1183.0,
  2311.0,
  2028.0,
  2098.0,
  2099.0,
  2335.0,
  2334.0,
  2336.0,
  2331.0],
 [2028.0, 2098.0, 2099.0, 2170.0, 2225.0, 2311.0])

In [19]:
results = []
for project_id in tqdm(reference_project_ids):
    if project_id in new_project_ids:
        df_reference = use_mask_project_id_based(df, project_id)
        df_newdata = use_mask_project_id_based(df_new_data, project_id)
        embedding_lst_reference = create_embedding_df(df_reference)
        embedding_lst_newdata = create_embedding_df(df_newdata)
        reference_df = pd.DataFrame(embedding_lst_reference).sample(300)
        newdata_df = pd.DataFrame(embedding_lst_newdata).sample(300)
        if len(reference_df) and len(newdata_df):
            data_drift_profile.calculate(
                reference_df, newdata_df,
                column_mapping=None
            )

            temp_dict = {}

            temp_dict["ref_project_id"] = project_id
            temp_dict["new_project_id"] = project_id
            data_drift_report_json = json.loads(data_drift_profile.json())
            temp_dict["n_features"] = data_drift_report_json['data_drift']['data']['metrics']['n_features']
            temp_dict["drift_features"] = data_drift_report_json['data_drift']['data']['metrics']['n_drifted_features']
            temp_dict["share_drift_features"] = data_drift_report_json['data_drift']['data']['metrics']['share_drifted_features']
            temp_dict["dataset_drift"] = data_drift_report_json['data_drift']['data']['metrics']['dataset_drift']

            results.append(temp_dict)

 55%|█████▍    | 6/11 [36:08<30:47, 369.59s/it]  

In [45]:
results

[{'ref_project_id': 2225.0,
  'new_project_id': 2225.0,
  'n_features': 768,
  'drift_features': 86,
  'share_drift_features': 0.11197916666666667,
  'dataset_drift': False},
 {'ref_project_id': 2170.0,
  'new_project_id': 2170.0,
  'n_features': 768,
  'drift_features': 42,
  'share_drift_features': 0.0546875,
  'dataset_drift': False},
 {'ref_project_id': 2311.0,
  'new_project_id': 2311.0,
  'n_features': 768,
  'drift_features': 14,
  'share_drift_features': 0.018229166666666668,
  'dataset_drift': False},
 {'ref_project_id': 2028.0,
  'new_project_id': 2028.0,
  'n_features': 768,
  'drift_features': 44,
  'share_drift_features': 0.057291666666666664,
  'dataset_drift': False},
 {'ref_project_id': 2098.0,
  'new_project_id': 2098.0,
  'n_features': 768,
  'drift_features': 416,
  'share_drift_features': 0.5416666666666666,
  'dataset_drift': True},
 {'ref_project_id': 2099.0,
  'new_project_id': 2099.0,
  'n_features': 768,
  'drift_features': 82,
  'share_drift_features': 0.10677

In [46]:
dftest = pd.DataFrame.from_records(results)

In [47]:
dftest

,ref_project_id,new_project_id,n_features,drift_features,share_drift_features,dataset_drift
0,2225.0,2225.0,768,86,0.111979,False
1,2170.0,2170.0,768,42,0.054688,False
2,2311.0,2311.0,768,14,0.018229,False
3,2028.0,2028.0,768,44,0.057292,False
4,2098.0,2098.0,768,416,0.541667,True
5,2099.0,2099.0,768,82,0.106771,False
